In [35]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [36]:
import tensorflow as tf

In [ ]:
# !pip install opencv-python --upgrade
# !pip uninstall opencv-python-headless -y 

### getting the dataset ready

In [ ]:
!tar -xf lfw.tgz

In [37]:
IMG_PATH=os.path.join('data','images')

#### negative images

In [ ]:
for directories in os.listdir('lfw'):
    for files in os.listdir(os.path.join('lfw',directories)):
        OLD_PATH=os.path.join('lfw',directories,files)
        NEW_PATH=os.path.join(IMG_PATH,files)
        os.replace(OLD_PATH,NEW_PATH)

#### user image

In [4]:
USER_PATH=os.path.join("data","user","face")

In [ ]:
## capturing image

cap=cv2.VideoCapture(0)

while cap.isOpened():
    ret,frame=cap.read()
    frame=frame[50:50+250,210:210+250]
    if cv2.waitKey(1) & 0xFF==ord('c'):
        imgname=os.path.join(USER_PATH,"user_face.jpg")
        cv2.imwrite(imgname,frame)
        break
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

#### augment the image for positives and anchor images

In [ ]:
import uuid

In [5]:
AUG_PATH=os.path.join("data","user","aug_data")

In [ ]:
for folders in os.listdir(AUG_PATH):
    for i in range(30):
        img=cv2.imread(os.path.join(USER_PATH,"user_face.jpg"))
        imgname=os.path.join(AUG_PATH,folders,str(uuid.uuid1())+".jpg")

        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_up_down(img,seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100),np.random.randint(100))) 

        cv2.imwrite(imgname,img.numpy())

### loading images in tf data pipeline

In [6]:
positive=tf.data.Dataset.list_files(os.path.join(AUG_PATH+"/positive/*.jpg"),shuffle=False)
anchor=tf.data.Dataset.list_files(os.path.join(AUG_PATH+"/anchor/*.jpg"),shuffle=False)
negative=tf.data.Dataset.list_files(os.path.join(IMG_PATH+"/*.jpg"),shuffle=False).take(30)

In [7]:
def load_image(x):
    byte_image=tf.io.read_file(x)
    img=tf.io.decode_jpeg(byte_image)
    img=tf.image.resize(img,(100,100))
    img=img/255.0
    return img

### labelling

In [8]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data=positives.concatenate(negatives)

In [ ]:
sample=data.as_numpy_iterator()

In [ ]:
example=sample.next()

In [ ]:
example

In [9]:
def preprocess(input_image,val_image,label):
    return (load_image(input_image),load_image(val_image),label)

In [ ]:
res=preprocess(*example)

In [ ]:
plt.imshow(res[1])

### Train and Test partition

In [10]:
#build data loader pipeline
data=data.map(preprocess)
data=data.cache()
data=data.shuffle(buffer_size=3000)

In [ ]:
data

In [11]:
train_data=data.take(round(len(data)*.7))
train_data=train_data.batch(8)
train_data=train_data.prefetch(4)

In [ ]:
train_data

In [12]:
test_data=data.skip(round(len(data)*.7))
test_data=test_data.take(round(len(data)*.3))
test_data=test_data.batch(8)
test_data=test_data.prefetch(4)

### building feature extractor(embedding layer)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Input,Conv2D,Dense,MaxPooling2D,Flatten

In [14]:
def feature_extractor():
    inp=Input(shape=(100,100,3),name="input_image")
    
    c1=Conv2D(64,(10,10),activation="relu")(inp)
    m1=MaxPooling2D(64,(2,2),padding="same")(c1)
    
    c2=Conv2D(128,(7,7),activation="relu")(m1)
    m2=MaxPooling2D(64,(2,2),padding="same")(c2)
    
    c3=Conv2D(128,(4,4),activation="relu")(m2)
    m3=MaxPooling2D(64,(2,2),padding="same")(c3)
    
    c4=Conv2D(256,(4,4),activation="relu")(m3)
    f1=Flatten()(c4)
    d1=Dense(4096,activation="sigmoid")(f1)
    
    return Model(inputs=[inp],outputs=[d1],name="feature_extractor")

In [15]:
featExt=feature_extractor()

In [16]:
featExt.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128) 

### build distance layer

In [17]:
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
        
    def call(self,input_embedding,val_embedding):
        return(tf.math.abs(input_embedding-val_embedding))

### siamese model

In [18]:
def siamese_model():
    input_image=Input(shape=(100,100,3),name="input_image")
    val_image=Input(shape=(100,100,3),name="val_image")
    
    siamese_layer=L1Dist()
    siamese_layer._name="l1_dist_layer"
    
    distance=siamese_layer(featExt(input_image),featExt(val_image))
    
    classifier=Dense(1,activation="sigmoid")(distance)
    
    return Model(inputs=[input_image,val_image],outputs=classifier,name="SiameseNetwork")

In [19]:
sm=siamese_model()

In [ ]:
sm.summary()

### Training

In [20]:
binary_cross_entropy=tf.losses.BinaryCrossentropy()

In [21]:
opt=tf.keras.optimizers.Adam(1e-4)

In [22]:
##checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=sm)

#### train step funtion

In [23]:
test_batch=train_data.as_numpy_iterator()

In [24]:
batch_1=test_batch.next()

In [ ]:
# batch_1

In [25]:
X=batch_1[:2]

In [ ]:
y=batch_1[2]

In [ ]:
y

In [26]:
def train_step(batch):
    
    with tf.GradientTape() as tape:
        X=batch[:2]
        y=batch[2] 
        
        yhat=sm(X,training=True)
        loss=binary_cross_entropy(y,yhat)
    print(loss)
    grad=tape.gradient(loss,sm.trainable_variables)
        
    opt.apply_gradients(zip(grad,sm.trainable_variables))
    
    return loss

#### training loop

In [27]:
from tensorflow.keras.metrics import Recall,Precision

In [28]:
def train(data,EPOCHS):
    for epoch in range(1,EPOCHS+1):
        print("\n Epoch {}/{}".format(epoch,EPOCHS))
        progbar=tf.keras.utils.Progbar(len(data))
        
        r=Recall()
        p=Precision()
        
        for idx,batch in enumerate(data):
            loss=train_step(batch)
            yhat=sm.predict(batch[:2])
            r.update_state(yhat,batch[2])
            p.update_state(yhat,batch[2])
            progbar.update(idx+1)
        print(loss.numpy(),r.result().numpy(),p.result().numpy())
        
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix) 

### Train the model

In [31]:
EPOCHS=15

In [32]:
train(train_data,EPOCHS)


 Epoch 1/15
tf.Tensor(0.69599676, shape=(), dtype=float32)
6/6 [==============================] - 90s 13s/step
0.2511413 0.42857143 1.0

 Epoch 2/15
tf.Tensor(0.26538214, shape=(), dtype=float32)
6/6 [==============================] - 80s 13s/step
0.7043867 0.47619048 1.0

 Epoch 3/15
tf.Tensor(0.30543986, shape=(), dtype=float32)
6/6 [==============================] - 78s 13s/step
0.75959796 0.42857143 1.0

 Epoch 4/15
tf.Tensor(0.17815566, shape=(), dtype=float32)
6/6 [==============================] - 90s 15s/step
0.46008325 0.45238096 1.0

 Epoch 5/15
tf.Tensor(0.45132747, shape=(), dtype=float32)
6/6 [==============================] - 82s 14s/step
0.04001529 0.47619048 1.0

 Epoch 6/15
tf.Tensor(0.33325195, shape=(), dtype=float32)
6/6 [==============================] - 98s 17s/step
0.6499585 0.52380955 1.0

 Epoch 7/15
tf.Tensor(0.37050128, shape=(), dtype=float32)
6/6 [==============================] - 111s 18s/step
0.20428587 0.5714286 1.0

 Epoch 8/15
tf.Tensor(0.3342268, sha

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()


In [ ]:
y_hat = sm.predict([test_input, test_val])

In [ ]:
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

In [ ]:
y_true

In [ ]:
plt.imshow(test_val[3])

### Save model

In [33]:
from tensorflow.keras.models import load_model

In [34]:
sm.save("siameseModel.h5")

In [ ]:
sm=load_model("siameseModel.h5",custom_objects={"L1Dist":L1Dist,"binary_cross_entropy":binary_cross_entropy})

In [ ]:
sm.compile(optimizer=tf.keras.optimizers.Adam(1e-4),loss=binary_cross_entropy)

### Realtime Detection

In [ ]:
# user_img.shape

In [ ]:
# plt.imshow(user_img)

In [ ]:
# img=cv2.imread("./data/user/val.jpeg")
# img=tf.image.resize(img,(100,100))
# img=img/255.0

In [ ]:
# sm.predict(list(np.expand_dims([img, user_img], axis=1)))

In [ ]:
## capturing image

cap=cv2.VideoCapture(0)

while cap.isOpened():
    ret,frame=cap.read()
    frame=frame[50:50+250,210:210+250]
   
    if cv2.waitKey(1) & 0xFF==ord('c'):
        print("processing...")
        imgname=os.path.join(USER_PATH,"user_face.jpg")
        cv2.imwrite(imgname,frame)
        break
            
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
batch=(list(np.expand_dims([user_img],axis=1)),list(np.expand_dims([user_img],axis=1)),np.asarray(1).astype('float32').reshape((-1,1)))

In [ ]:
train_step(batch)

In [ ]:
user_img=cv2.imread(os.path.join(USER_PATH,"user_face.jpg"))
user_img=tf.image.resize(user_img,(100,100))
user_img=user_img/255.0

In [ ]:
cap=cv2.VideoCapture(0)

while cap.isOpened():
    ret,frame=cap.read()
    frame=frame[50:50+250,210:210+250,:]
    
    try:
        if cv2.waitKey(1) & 0xFF==ord('v'):
            print("verifying")
            img=tf.image.resize(frame,(100,100))
            img=img/255.0
            results=sm.predict(list(np.expand_dims([img, user_img], axis=1)),verbose=False)
            print(results)
            
            if results>0.5:
                print("Access Granted!")
            else:
                print("Access Denied")
        # print(img.shape)
    except Exception as e:
        print(e)
            
    cv2.imshow("frame",frame)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()